In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 13s 0us/step


In [5]:
print('Shape of the Dataset')
print('==================================')
print(f'X_train Shape: {X_train.shape}')
print('-------------------------------------')
print(f'X_test Shape: {X_test.shape}')
print('-------------------------------------')
print(f'Y_train Shape: {y_train.shape}')
print('-------------------------------------')
print(f'Y_test Shape: {y_test.shape}')

Shape of the Dataset
X_train Shape: (50000, 32, 32, 3)
-------------------------------------
X_test Shape: (10000, 32, 32, 3)
-------------------------------------
Y_train Shape: (50000, 1)
-------------------------------------
Y_test Shape: (10000, 1)


In [6]:
from keras.applications.resnet50 import preprocess_input

def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_images = preprocess_input(input_images)
  return output_images

X_train_prep = preprocess_image_input(X_train)
X_test_prep = preprocess_image_input(X_test)

In [7]:
from keras.applications.resnet import ResNet50
from keras.layers import GlobalAveragePooling2D, Flatten, Dense, UpSampling2D, Input
from keras import Model

def feature_extractor(inputs):

  feature_extractor = ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')(inputs)

  return feature_extractor

def classifier(inputs):
    x = GlobalAveragePooling2D()(inputs)
    x = Flatten()(x)
    x = Dense(1024, activation="relu")(x)
    x = Dense(512, activation="relu")(x)
    x = Dense(10, activation="softmax")(x)
    return x

def final_model(inputs):

    resize = UpSampling2D(size=(7,7))(inputs)

    resnet_feature_extractor = feature_extractor(resize)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output

def define_compile_model():
  inputs = Input(shape=(32, 32, 3))

  classification_output = final_model(inputs)
  model = Model(inputs=inputs, outputs = classification_output)

  model.compile(optimizer='SGD',
                loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])

  return model


model = define_compile_model()

model.summary()

94765736/94765736 [==============================] - 5s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 up_sampling2d (UpSampling2  (None, 224, 224, 3)       0         
 D)                                                              
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                              

In [8]:
EPOCHS = 4
history = model.fit(X_train_prep, y_train, epochs=EPOCHS, validation_data = (X_test_prep, y_test), batch_size=64)

Epoch 1/4
782/782 [==============================] - 614s 751ms/step - loss: 0.4028 - accuracy: 0.8686 - val_loss: 0.2462 - val_accuracy: 0.9177
Epoch 2/4
782/782 [==============================] - 584s 747ms/step - loss: 0.1005 - accuracy: 0.9673 - val_loss: 0.1768 - val_accuracy: 0.9416
Epoch 3/4
782/782 [==============================] - 584s 747ms/step - loss: 0.0328 - accuracy: 0.9908 - val_loss: 0.1919 - val_accuracy: 0.9411
Epoch 4/4
782/782 [==============================] - 574s 734ms/step - loss: 0.0136 - accuracy: 0.9969 - val_loss: 0.1663 - val_accuracy: 0.9524


In [9]:
test_loss, test_acc = model.evaluate(X_test_prep, y_test, batch_size=64)
print(f'Accuracy: {test_acc}')

157/157 [==============================] - 31s 198ms/step - loss: 0.1663 - accuracy: 0.9524
Accuracy: 0.9524000287055969


In [10]:
model.save('ResNet50_model.keras')

In [11]:
from google.colab import files

files.download('ResNet50_model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>